# Project Pythia Notebook Template

Let's start here! If you can directly link to an image relevant to your notebook, such as [canonical logos](https://github.com/numpy/numpy/blob/main/doc/source/_static/numpylogo.svg), do so here at the top of your notebook. You can do this with MyST Markdown syntax, outlined in [this MyST guide](https://mystmd.org/guide/figures), or you edit this cell to see a demonstration. **Be sure to include `alt` text for any embedded images to make your content more accessible.**

```{image} ../thumbnails/thumbnail.png
:alt: Project Pythia logo
:width: 200px
```

Next, title your notebook appropriately with a top-level Markdown header, `#` (see the very first cell above). Do not use this level header anywhere else in the notebook. Our book build process will use this title in the navbar, table of contents, etc. Keep it short, keep it descriptive. 

Follow this with a `---` cell to visually distinguish the transition to the prerequisites section.

---

## Overview
If you have an introductory paragraph, lead with it here! Keep it short and tied to your material, then be sure to continue into the required list of topics below,

1. This is a numbered list of the specific topics
1. These should map approximately to your main sections of content
1. Or each second-level, `##`, header in your notebook
1. Keep the size and scope of your notebook in check
1. And be sure to let the reader know up front the important concepts they'll be leaving with

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total. Safer to round up and overestimate.
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports
Begin your body of content with another `---` divider before continuing into this section, then remove this body text and populate the following code cell with all necessary Python imports **up-front**:

In [ ]:
import xarray as xr
import s3fs
import pelicanfs as pefs

## Initializing the datasets

This is where you begin your first section of material, loosely tied to your objectives stated up front. Tie together your notebook as a narrative, with interspersed Markdown text, images, and more as necessary,

In [ ]:
def init_datasets():
    bucket_name = 'noaa-wcsd-zarr-pds'
    ship_name = "Henry_B._Bigelow"
    cruise_name = "HB1906"
    sensor_name = "EK60"

    s3_file_system = s3fs.S3FileSystem(anon=True)
    zarr_store = f'{cruise_name}.zarr'
    s3_zarr_store_path = f"{bucket_name}/level_2/{ship_name}/{cruise_name}/{sensor_name}/{zarr_store}"
    store = s3fs.S3Map(root=s3_zarr_store_path, s3=s3_file_system, check=False)
    cruise = xr.open_zarr(store=store, consolidated=None)
    start_time = "2019-10-16T15:00:00"
    end_time = "2019-10-16T23:30:00"
    timeslice = slice(start_time, end_time)
    #
    cruise = cruise.sel(time=timeslice, drop=False)
    cruise = cruise.sel(frequency=38000, method='nearest')
    print(cruise)

    # location of one specific buoy located on Georges Bank
    target_lon = 360 - 66.546
    target_lat = 41.088
    print(f"Target coordinates: Longitude: {target_lon}, Latitude: {target_lat}")

    bouy_data_day_before = xr.open_dataset(
        'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191012.nc#mode=bytes', engine='netcdf4')
    buoy_data_actual_day = xr.open_dataset(
        'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191013.nc#mode=bytes',
        engine='netcdf4')
    buoy_data_day_after = xr.open_dataset(
        'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191014.nc#mode=bytes',
        engine='netcdf4')
    sst_day_before = bouy_data_day_before['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
    sst_actual_day = buoy_data_actual_day['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
    sst_day_after = buoy_data_day_after['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]

    return sst_day_before, sst_actual_day, sst_day_after, cruise

### Data Preprocessing
Keep up the good work! A note, *try to avoid using code comments as narrative*, and instead let them only exist as brief clarifications where necessary.

In [ ]:
# baseline_timestamps = cruise.time.values
# sst = np.full(shape=baseline_timestamps.shape, fill_value=np.nan)
# df = pd.DataFrame(sst, columns=['sst'], index=baseline_timestamps)
# df.loc[df.index[0],'sst'] = sst_day_before
# df.loc[df.index[-1],'sst'] = sst_day_after
# # print(int((len(df.index)/2)))
# df.loc[df.index[int((len(df.index)/2))],'sst'] = sst_actual_day
# df = df.interpolate(method='time')
# print(df)

---